In [1]:
import os

In [7]:
import geopandas as gpd

C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [145]:
filep = 'corrected/S2B_MSIL2A_20201108T054039_N9999_R005_T43SFR_20210922T185740.SAFE/GRANULE/L2A_T43SFR_A019193_20201108T055005/QI_DATA'

In [146]:
cmd = f'ogr2ogr -f "ESRI Shapefile" cloud_mask/L1C_T43SFR_A019193_20201108T055005/cloud_mask.shp {filep}/MSK_CLOUDS_B00.gml'

In [147]:
os.system(cmd)

1

In [5]:
img_dir = 'N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/'

In [6]:
train_dir = img_dir + 'train_area_sample/'

In [39]:
cloud_mask = gpd.read_file('./cloud_mask/cloud_mask.shp')

In [18]:
cloud_mask

,gml_id,maskType,geometry
0,OPAQUE.0,OPAQUE,"POLYGON ((613020.000 3555060.000, 613320.000 3..."
1,OPAQUE.1,OPAQUE,"POLYGON ((609780.000 3552840.000, 609960.000 3..."
2,OPAQUE.2,OPAQUE,"POLYGON ((662340.000 3550860.000, 662580.000 3..."
3,OPAQUE.3,OPAQUE,"POLYGON ((611580.000 3549600.000, 611820.000 3..."
4,OPAQUE.4,OPAQUE,"POLYGON ((612060.000 3549360.000, 612300.000 3..."
...,...,...,...
265,CIRRUS.19,CIRRUS,"POLYGON ((670620.000 3538980.000, 670800.000 3..."
266,CIRRUS.20,CIRRUS,"POLYGON ((657540.000 3533700.000, 657720.000 3..."
267,CIRRUS.21,CIRRUS,"POLYGON ((653520.000 3533640.000, 653700.000 3..."
268,CIRRUS.22,CIRRUS,"POLYGON ((663060.000 3532680.000, 663240.000 3..."


In [53]:
cloud_mask_proj

,gml_id,maskType,geometry
0,OPAQUE.0,OPAQUE,"POLYGON ((76.19815 32.12627, 76.20133 32.12624..."
1,OPAQUE.1,OPAQUE,"POLYGON ((76.16356 32.10656, 76.16546 32.10655..."
2,OPAQUE.2,OPAQUE,"POLYGON ((76.72012 32.08237, 76.72266 32.08233..."
3,OPAQUE.3,OPAQUE,"POLYGON ((76.18225 32.07716, 76.18480 32.07714..."
4,OPAQUE.4,OPAQUE,"POLYGON ((76.18731 32.07495, 76.18985 32.07493..."
...,...,...,...
265,CIRRUS.19,CIRRUS,"POLYGON ((76.80571 31.97402, 76.80762 31.97399..."
266,CIRRUS.20,CIRRUS,"POLYGON ((76.66648 31.92830, 76.66838 31.92827..."
267,CIRRUS.21,CIRRUS,"POLYGON ((76.62396 31.92831, 76.62586 31.92828..."
268,CIRRUS.22,CIRRUS,"POLYGON ((76.72468 31.91832, 76.72658 31.91829..."


In [19]:
cloud_mask.crs

<Projected CRS: EPSG:32643>
Name: WGS 84 / UTM zone 43N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 72°E and 78°E, northern hemisphere between equator and 84°N, onshore and offshore. China. India. Kazakhstan. Kyrgyzstan. Maldives. Pakistan. Russian Federation. Tajikistan.
- bounds: (72.0, 0.0, 78.0, 84.0)
Coordinate Operation:
- name: UTM zone 43N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
import os
import re
import sys
import math
import fiona
import pickle
import logging
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import skimage
import skimage.draw
import pyproj
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [34]:
def load_target_shp(path, transform=None, proj_out=None):
    """ Load the shapefile as a list of numpy array of coordinates
        INPUT : path (str) -> the path to the shapefile
                transform (rasterio.Affine) -> the affine transformation to get the polygon in row;col format from UTM.
        OUTPUT : poly (list of np.array) -> list of polygons (as numpy.array of coordinates)
                 poly_rc (list of np.array) -> list of polygon in row-col format if a transform is given
    """
    print("Loading target shapefile...")
    with fiona.open(path) as shapefile:
        proj_in = pyproj.Proj(shapefile.crs)
        class_type = [1 for feature in shapefile]
        features = [feature["geometry"] for feature in shapefile]
    # re-project polygons if necessary
    if proj_out is None or proj_in == proj_out:
        poly = [np.array([(coord[0], coord[1]) for coord in features[i]['coordinates'][0]]) for i in
                range(len(features))]
        print('No re-projection!')
    else:
        poly = [np.array(
            [pyproj.transform(proj_in, proj_out, coord[0], coord[1]) for coord in features[i]['coordinates'][0]]) for i
            in range(len(features))]
        print(f'Re-project from {proj_in} to {proj_out}')

    poly_rc = None
    # transform in row-col if a transform is given
    if transform is not None:
        poly_rc = [np.array([rasterio.transform.rowcol(transform, coord[0], coord[1])[::-1] for coord in p]) for p in
                   poly]
    print('Loaded target shape files.')

    return features, poly, poly_rc, class_type


def compute_mask(polygon_list, meta, val_list):
    """ Get mask of class of a polygon list
        INPUT : polygon_list (list od polygon in coordinates (x, y)) -> the polygons in row;col format
                meta -> the image width and height
                val_list(list of int) -> the class associated with each polygon
        OUTPUT : img (np.array 2D) -> the mask in which the pixel value reflect it's class (zero being the absence of class)
    """
    img = np.zeros((meta['height'], meta['width']), dtype=np.uint8)  # skimage : row,col --> h,w
    i = 0
    for polygon, val in zip(polygon_list, val_list):
        rr, cc = skimage.draw.polygon(polygon[:, 1], polygon[:, 0], img.shape)
        img[rr, cc] = val
        i += 1
    print("Added targets' mask.")
    return img


def load_geotiff(path, window=None):
    """ Load the geotiff as a list of numpy array.
        INPUT : path (str) -> the path to the geotiff
                window (rasterio.windows.Window) -> the window to use when loading the image
        OUTPUT : band (list of numpy array) -> the different bands as float scaled to 0:1
                 meta (dictionary) -> the metadata associated with the geotiff
    """
    with rasterio.open(path) as f:
        band = [skimage.img_as_float(f.read(i + 1, window=window)) for i in range(f.count)]
        meta = f.meta
        if window is not None:
            meta['height'] = window.height
            meta['width'] = window.width
            meta['transform'] = f.window_transform(window)
    return band, meta

In [36]:
eg = img_dir + 'train_area/L2A_T43SFR_A026271_20200703T053446.tiff'
band, meta = load_geotiff(eg)

In [37]:
features, poly, poly_rc, class_type = load_target_shp('./cloud_mask/cloud_mask.shp', meta['transform'], pyproj.Proj(meta['crs']))

Loading target shapefile...


C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lida\AppData\Local\Temp/ipykernel_17832/2398201670.py:20: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  [pyproj.transform(proj_in, proj_out, coord[0], coord[1]) for coord in features[i]['coordinates'][0]]) for i


KeyboardInterrupt: 

In [38]:
train_area_shp = gpd.read_file('../data/train_area/train_area.shp')

In [54]:
train_area_shp['geometry'].overlay(cloud_mask_proj, how='intersection')

AttributeError: 'GeoSeries' object has no attribute 'overlay'

In [69]:
df = train_area_shp.geometry.intersection(cloud_mask_proj.geometry)

C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\geopandas\base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [85]:
df.is_empty

0       True
1      False
2      False
3      False
4      False
       ...  
265    False
266    False
267    False
268    False
269    False
Length: 270, dtype: bool

In [87]:
df == None

0      False
1       True
2       True
3       True
4       True
       ...  
265     True
266     True
267     True
268     True
269     True
Length: 270, dtype: bool

In [96]:
a = df[(df != None) & (~df.is_empty)]

In [101]:
a

GeoSeries([], dtype: geometry)

In [102]:
a.shape 

(0,)

In [66]:
dff = train_area_shp.sjoin(cloud_mask_proj, how='intersection')

AttributeError: 'GeoDataFrame' object has no attribute 'sjoin'

In [56]:
train_area_shp

,id,geometry
0,0,"POLYGON ((77.17355 32.23500, 77.20571 32.25445..."


In [46]:
train_area_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [50]:
meta['crs']

CRS.from_epsg(4326)

In [51]:
cloud_mask_proj = cloud_mask.to_crs(meta['crs'])

In [ ]:
cloud_mask_proj.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [58]:
from shapely.geometry import Polygon
polys1 = gpd.GeoSeries([Polygon([(0,0), (2,0), (2,2), (0,2)]),
                              Polygon([(2,2), (4,2), (4,4), (2,4)])])
polys2 = gpd.GeoSeries([Polygon([(1,1), (3,1), (3,3), (1,3)]),
                              Polygon([(3,3), (5,3), (5,5), (3,5)])])
df1 = gpd.GeoDataFrame({'geometry': polys1, 'df1_data':[1,2]})
df2 = gpd.GeoDataFrame({'geometry': polys2, 'df2_data':[1,2]})

In [61]:
df1.intersection(df2)

0    POLYGON ((2.00000 2.00000, 2.00000 1.00000, 1....
1    POLYGON ((4.00000 4.00000, 4.00000 3.00000, 3....
dtype: geometry

In [67]:
df1.sjoin(df2, how='intersection')

AttributeError: 'GeoDataFrame' object has no attribute 'sjoin'

### build cloud_mask from gml to shp

In [122]:
def convert_gml_to_shp(img_dir):
    safe_dir = img_dir + 'safe/'
    cloud_dir = img_dir + 'cloud_mask/'
    if not os.path.exists(cloud_dir):
        os.mkdir(cloud_dir)

    safe_names = os.listdir(safe_dir)

    print('Converting cloud mask from gml to shp ...')
    for i, safe_name in enumerate(safe_names, start=1):
        filename = os.listdir(safe_dir + safe_name + '/GRANULE/')[0]
        gml_path = safe_dir + safe_name + '/GRANULE/' + filename + '/QI_DATA/MSK_CLOUDS_B00.gml'
        if not os.path.exists(cloud_dir + filename):
            os.mkdir(cloud_dir + filename)
        cloud_path = cloud_dir + filename + '/cloud_mask.shp'
        cmd = f'ogr2ogr -f "ESRI Shapefile" {cloud_path} {gml_path}'
        returned = os.system(cmd)
        if returned == 0:
            print(f'[{i}/{len(safe_names)}] {cloud_path} ')
        else:
            raise ValueError(f'{safe_name} failed')
    print(f'  ok')

In [123]:
convert_gml_to_shp(img_dir)

Converting cloud mask from gml to shp ...
[1/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A023697_20200105T053922/cloud_mask.shp 
[2/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A023740_20200108T054215/cloud_mask.shp 
[3/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A023840_20200115T053150/cloud_mask.shp 
[4/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A023883_20200118T054145/cloud_mask.shp 
[5/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A023983_20200125T053846/cloud_mask.shp 
[6/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A024026_20200128T054632/cloud_mask.shp 
[7/131] N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/cloud_mask/L1C_T43SFR_A024126_20200204T053431/cloud_mask.shp 
[8/131] N:/dataorg-datasets/MLsate

In [124]:
filepath = img_dir + 'safe/S2B_MSIL1C_20201225T053229_N0209_R105_T43SFR_20201225T064223.SAFE/GRANULE/L1C_T43SFR_A019865_20201225T053231/IMG_DATA/T43SFR_20201225T053229_B02.jp2'
with rasterio.open(filepath) as src:
    b = src.read(1)

In [126]:
b

array([[   0,    0,    0, ..., 8016, 7260, 7454],
       [   0,    0,    0, ..., 8102, 7893, 7812],
       [   0,    0,    0, ..., 7658, 7942, 7851],
       ...,
       [   0,    0,    0, ...,  644,  652,  661],
       [   0,    0,    0, ...,  655,  666,  658],
       [   0,    0,    0, ...,  652,  652,  674]], dtype=uint16)

In [128]:
c = gpd.read_file(img_dir + 'cloud_mask/L1C_T43SFR_A014860_20200110T053549/cloud_mask.shp')

In [132]:
c

,gml_id,maskType,geometry
0,OPAQUE.0,OPAQUE,"POLYGON ((634260.000 3600000.000, 634560.000 3..."
1,OPAQUE.1,OPAQUE,"POLYGON ((639900.000 3600000.000, 640860.000 3..."
2,OPAQUE.2,OPAQUE,"POLYGON ((634920.000 3599580.000, 635100.000 3..."
3,OPAQUE.3,OPAQUE,"POLYGON ((636000.000 3599340.000, 636240.000 3..."
4,OPAQUE.4,OPAQUE,"POLYGON ((643320.000 3598740.000, 643500.000 3..."
...,...,...,...
702,CIRRUS.411,CIRRUS,"POLYGON ((670620.000 3490800.000, 670800.000 3..."
703,CIRRUS.412,CIRRUS,"POLYGON ((702960.000 3490740.000, 703140.000 3..."
704,CIRRUS.413,CIRRUS,"POLYGON ((672480.000 3493500.000, 672720.000 3..."
705,CIRRUS.414,CIRRUS,"POLYGON ((697380.000 3491100.000, 697560.000 3..."


In [135]:
cloud_legend = {'OPAQUE': 1, 'CIRRUS': 2}

In [137]:
import shapely

In [138]:
shapes = iter([(shapely.geometry.mapping(poly), cloud_legend[v]) for poly, v in zip(c.geometry, c.maskType)])
cloud_img = rasterio.features.rasterize(shapes, out_shape=(meta['height'], meta['width']),
                                       transform=meta['transform'])

In [140]:
cloud_img.shape

(2357, 1892)

In [141]:
np.unique(cloud_img)

array([0], dtype=uint8)

In [144]:
'L2A'+'L1C_T43SFR_A014760_20200103T054832'.lstrip('L1C')

'L2A_T43SFR_A014760_20200103T054832'

In [159]:
a = gpd.GeoDataFrame([{'gml_id': 'CLOUDLESS.0', 'maskType': 'CLOUDLESS', 'geometry': shapely.geometry.Polygon()}])

In [160]:
a

,gml_id,maskType,geometry
0,CLOUDLESS.0,CLOUDLESS,GEOMETRYCOLLECTION EMPTY


In [161]:
a.to_file('out.shp', driver='ESRI Shapefile')

DriverIOError: Geometry type of 'Geometry Collection' not supported in shapefiles.  Type can be overridden with a layer creation option of SHPT=POINT/ARC/POLYGON/MULTIPOINT/POINTZ/ARCZ/POLYGONZ/MULTIPOINTZ/MULTIPATCH.

In [163]:
eg_band, eg_meta = load_geotiff(img_dir + 'geotiff/L2A_T43SFR_A014760_20200103T054832.tiff')

In [164]:
eg_meta

{'driver': 'JP2OpenJPEG',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 5,
 'crs': None,
 'transform': Affine(10.0, 0.0, 600000.0,
        0.0, -10.0, 3600000.0)}

In [ ]:
eg_band[-1]